# Deep Learning
# Assignment 3

Previously in 2_fullyconnected.ipynb, you trained a logistic regression and a neural network model.
The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in 1_notmnist.ipynb.

In [19]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:

* data as a flat matrix,
* labels as float 1-hot encodings.

In [20]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

# Problem 1
Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor t using nn.l2_loss(t). The right amount of regularization should improve your validation / test accuracy.

In [17]:
# l2 loss in the n.n model
graph = tf.Graph()
h1_size = 1024
batch_size = 128
beta = 0.01
with graph.as_default():
    # data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # weights
    weights = {
        'h1': tf.Variable(tf.truncated_normal([image_size*image_size, h1_size])),
        'out': tf.Variable(tf.truncated_normal([h1_size, num_labels]))
    }
    
    biases = {
        'h1': tf.Variable(tf.zeros([h1_size])),
        'out': tf.Variable(tf.zeros([num_labels]))
    }
    
    # model
    def mlp(x,weights,biases):
        layer_1 = tf.matmul(x,weights['h1']) + biases['h1']
        layer_1 = tf.nn.relu(layer_1)
        
        layer_out = tf.matmul(layer_1, weights['out']) + biases['out']
        return layer_out
    
    # loss
    logits = mlp(tf_train_dataset, weights, biases)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))+ beta*tf.nn.l2_loss(
        weights['h1'])+ beta*tf.nn.l2_loss(weights['out'])
    
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = mlp(tf_valid_dataset, weights, biases)
    test_prediction = mlp(tf_test_dataset, weights, biases)
    
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for step in range(801):
        offset = np.random.randint(0, train_dataset.shape[0]-batch_size)
        batch_data = train_dataset[offset:(offset+batch_size), :]
        batch_label = train_labels[offset:(offset+batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_label}
        _,l,pred = session.run([optimizer,loss,train_prediction],feed_dict= feed_dict)
        if step%100 ==0:
            pred_v = valid_prediction.eval()
            print('step: {}, loss: {}, train_accuracy: {}, valid_accuracy: {}'.format(step, l, accuracy(pred,batch_label), 
                                                                                     accuracy(pred_v,valid_labels)))
    pred_t = test_prediction.eval()
    print('Training Done!')
    print(accuracy(pred_t,test_labels))
    

step: 0, loss: 3445.98974609375, train_accuracy: 10.15625, valid_accuracy: 31.89
step: 100, loss: 1154.400390625, train_accuracy: 82.03125, valid_accuracy: 78.6
step: 200, loss: 420.4654846191406, train_accuracy: 75.0, valid_accuracy: 80.51
step: 300, loss: 153.6602783203125, train_accuracy: 85.15625, valid_accuracy: 82.42
step: 400, loss: 56.89268112182617, train_accuracy: 80.46875, valid_accuracy: 83.93
step: 500, loss: 21.329565048217773, train_accuracy: 85.15625, valid_accuracy: 83.89
step: 600, loss: 8.345946311950684, train_accuracy: 82.03125, valid_accuracy: 83.51
step: 700, loss: 3.649322032928467, train_accuracy: 85.15625, valid_accuracy: 83.59
step: 800, loss: 1.8136191368103027, train_accuracy: 82.03125, valid_accuracy: 83.09
Training Done!
90.04


# Problem 2
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

In [18]:
train_dataset = train_dataset[:1280]
train_labels = train_labels[:1280]
# l2 loss in the n.n model
graph = tf.Graph()
h1_size = 1024
batch_size = 128
beta = 0.01
with graph.as_default():
    # data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # weights
    weights = {
        'h1': tf.Variable(tf.truncated_normal([image_size*image_size, h1_size])),
        'out': tf.Variable(tf.truncated_normal([h1_size, num_labels]))
    }
    
    biases = {
        'h1': tf.Variable(tf.zeros([h1_size])),
        'out': tf.Variable(tf.zeros([num_labels]))
    }
    
    # model
    def mlp(x,weights,biases):
        layer_1 = tf.matmul(x,weights['h1']) + biases['h1']
        layer_1 = tf.nn.relu(layer_1)
        
        layer_out = tf.matmul(layer_1, weights['out']) + biases['out']
        return layer_out
    
    # loss
    logits = mlp(tf_train_dataset, weights, biases)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))+ beta*tf.nn.l2_loss(
        weights['h1'])+ beta*tf.nn.l2_loss(weights['out'])
    
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = mlp(tf_valid_dataset, weights, biases)
    test_prediction = mlp(tf_test_dataset, weights, biases)
    
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for step in range(801):
        offset = np.random.randint(0, train_dataset.shape[0]-batch_size)
        batch_data = train_dataset[offset:(offset+batch_size), :]
        batch_label = train_labels[offset:(offset+batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_label}
        _,l,pred = session.run([optimizer,loss,train_prediction],feed_dict= feed_dict)
        if step%100 ==0:
            pred_v = valid_prediction.eval()
            print('step: {}, loss: {}, train_accuracy: {}, valid_accuracy: {}'.format(step, l, accuracy(pred,batch_label), 
                                                                                     accuracy(pred_v,valid_labels)))
    pred_t = test_prediction.eval()
    print('Training Done!')
    print(accuracy(pred_t,test_labels))
    

step: 0, loss: 3524.120361328125, train_accuracy: 15.625, valid_accuracy: 30.94
step: 100, loss: 1150.1893310546875, train_accuracy: 100.0, valid_accuracy: 78.01
step: 200, loss: 422.1091003417969, train_accuracy: 100.0, valid_accuracy: 78.09
step: 300, loss: 154.90679931640625, train_accuracy: 100.0, valid_accuracy: 78.26
step: 400, loss: 56.91923904418945, train_accuracy: 100.0, valid_accuracy: 79.37
step: 500, loss: 21.178165435791016, train_accuracy: 98.4375, valid_accuracy: 80.49
step: 600, loss: 8.05430793762207, train_accuracy: 100.0, valid_accuracy: 80.05
step: 700, loss: 3.2830681800842285, train_accuracy: 99.21875, valid_accuracy: 80.87
step: 800, loss: 1.478226900100708, train_accuracy: 100.0, valid_accuracy: 80.53
Training Done!
87.85


# Problem 3
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides nn.dropout() for that, but you have to make sure it's only inserted during training.
What happens to our extreme overfitting case?

In [22]:
# make sure training set is reloaded, expect the train size to be around 200000
print(train_dataset.shape, train_labels.shape)

(200000, 784) (200000, 10)


In [71]:
# l2 loss in the n.n model
graph = tf.Graph()
h1_size = 1024
batch_size = 128
beta = 0.01
with graph.as_default():
    # data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # weights
    weights = {
        'h1': tf.Variable(tf.truncated_normal([image_size*image_size, h1_size])),
        'out': tf.Variable(tf.truncated_normal([h1_size, num_labels]))
    }
    
    biases = {
        'h1': tf.Variable(tf.zeros([h1_size])),
        'out': tf.Variable(tf.zeros([num_labels]))
    }
    keep_prob = tf.placeholder(tf.float32)
    # model
    def mlp(x,weights,biases,keep_prob):
        layer_1 = tf.matmul(x,weights['h1']) + biases['h1']
        layer_1 = tf.nn.relu(layer_1)
        layer_1_drop = tf.nn.dropout(layer_1,keep_prob)
        
        layer_out = tf.matmul(layer_1_drop, weights['out']) + biases['out']
        return layer_out
    
    # loss
    logits = mlp(tf_train_dataset, weights, biases, keep_prob)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))+ beta*tf.nn.l2_loss(
        weights['h1'])+ beta*tf.nn.l2_loss(weights['out'])
    
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = mlp(tf_valid_dataset, weights, biases, tf.constant(1.))
    test_prediction = mlp(tf_test_dataset, weights, biases,tf.constant(1.))
    
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for step in range(801):
        offset = np.random.randint(0, train_dataset.shape[0]-batch_size)
        batch_data = train_dataset[offset:(offset+batch_size), :]
        batch_label = train_labels[offset:(offset+batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_label, keep_prob: 0.5}
        _,l,pred = session.run([optimizer,loss,train_prediction],feed_dict= feed_dict)
        if step%100 ==0:
            pred_v = valid_prediction.eval()
            print('step: {}, loss: {}, train_accuracy: {}, valid_accuracy: {}'.format(step, l, accuracy(pred,batch_label), 
                                                                                     accuracy(pred_v,valid_labels)))
    pred_t = test_prediction.eval()
    print('Training Done!')
    print(accuracy(pred_t,test_labels))
    

step: 0, loss: 3617.685546875, train_accuracy: 11.71875, valid_accuracy: 26.52
step: 100, loss: 1191.0137939453125, train_accuracy: 68.75, valid_accuracy: 75.08
step: 200, loss: 426.26715087890625, train_accuracy: 78.90625, valid_accuracy: 80.7
step: 300, loss: 155.64230346679688, train_accuracy: 75.78125, valid_accuracy: 82.27
step: 400, loss: 57.28995132446289, train_accuracy: 79.6875, valid_accuracy: 83.3
step: 500, loss: 21.474323272705078, train_accuracy: 82.03125, valid_accuracy: 83.63
step: 600, loss: 8.43882942199707, train_accuracy: 81.25, valid_accuracy: 82.67
step: 700, loss: 3.566711664199829, train_accuracy: 82.8125, valid_accuracy: 82.29
step: 800, loss: 1.9077297449111938, train_accuracy: 78.125, valid_accuracy: 83.0
Training Done!
89.67


# Problem 4
Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is 97.1%.

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

>global_step = tf.Variable(0)  # count the number of steps taken.

> learning_rate = tf.train.exponential_decay(0.5, global_step, ...)

> optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

In [74]:
# l2 loss in the n.n model
graph = tf.Graph()
h1_size = 512
h2_size = 1024
h3_size = 1024
batch_size = 128
beta = 0.
with graph.as_default():
    # data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # weights
    weights = {
        'h1': tf.Variable(tf.truncated_normal([image_size*image_size, h1_size])),
        'h2': tf.Variable(tf.truncated_normal([h1_size, h2_size])),
        'h3': tf.Variable(tf.truncated_normal([h2_size, h3_size])),
        'out': tf.Variable(tf.truncated_normal([h3_size, num_labels]))
    }
    
    biases = {
        'h1': tf.Variable(tf.zeros([h1_size])),
        'h2': tf.Variable(tf.zeros([h2_size])),
        'h3': tf.Variable(tf.zeros([h3_size])),
        'out': tf.Variable(tf.zeros([num_labels]))
    }
    keep_prob = tf.placeholder(tf.float32)

    # model
    def mlp(x,weights,biases,keep_prob):
        layer_1 = tf.matmul(x,weights['h1']) + biases['h1']
        layer_1 = tf.nn.relu(layer_1)
        layer_1_drop = tf.nn.dropout(layer_1,keep_prob)
        
        layer_2 = tf.matmul(layer_1,weights['h2']) + biases['h2']
        layer_2 = tf.nn.relu(layer_2)
        layer_2_drop = tf.nn.dropout(layer_2,keep_prob)
        
        layer_3 = tf.matmul(layer_2_drop,weights['h3']) + biases['h3']
        layer_3 = tf.nn.relu(layer_3)
        layer_3_drop = tf.nn.dropout(layer_3,keep_prob)
        
        layer_out = tf.matmul(layer_3_drop, weights['out']) + biases['out']
        return layer_out
    
    # loss
    logits = mlp(tf_train_dataset, weights, biases, 0.5)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))+ beta*tf.nn.l2_loss(
        weights['h1'])+ beta*tf.nn.l2_loss(weights['out']) + beta*tf.nn.l2_loss(weights['h2'])
    
    # optimizer
    optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss)
    
    # predictions
    # train_prediction = tf.nn.softmax(logits)
    train_prediction = mlp(tf_train_dataset, weights, biases, tf.constant(1.))
    valid_prediction = mlp(tf_valid_dataset, weights, biases, tf.constant(1.))
    test_prediction = mlp(tf_test_dataset, weights, biases,tf.constant(1.))
    
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for step in range(3501):
        offset = np.random.randint(0, train_dataset.shape[0]-batch_size)
        batch_data = train_dataset[offset:(offset+batch_size), :]
        batch_label = train_labels[offset:(offset+batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_label, keep_prob: 1}
        _,l,pred = session.run([optimizer,loss,train_prediction],feed_dict= feed_dict)
        if step%100 ==0:
            pred_v = valid_prediction.eval()
            print('step: {}, loss: {}, train_accuracy: {}, valid_accuracy: {}'.format(step, l, accuracy(pred,batch_label), 
                                                                                     accuracy(pred_v,valid_labels)))
    pred_t = test_prediction.eval()
    print('Training Done!')
    print(accuracy(pred_t,test_labels))
    

step: 0, loss: 185589.875, train_accuracy: 6.25, valid_accuracy: 7.48
step: 100, loss: 139109.34375, train_accuracy: 28.90625, valid_accuracy: 27.36
step: 200, loss: 111384.9296875, train_accuracy: 38.28125, valid_accuracy: 45.29
step: 300, loss: 79146.1875, train_accuracy: 64.0625, valid_accuracy: 57.31
step: 400, loss: 78234.71875, train_accuracy: 70.3125, valid_accuracy: 64.26
step: 500, loss: 71399.15625, train_accuracy: 64.84375, valid_accuracy: 67.35
step: 600, loss: 58080.66796875, train_accuracy: 73.4375, valid_accuracy: 69.9
step: 700, loss: 52651.2890625, train_accuracy: 73.4375, valid_accuracy: 71.32


KeyboardInterrupt: 